<a href="https://colab.research.google.com/github/chakrateja70/claude-Agent/blob/main/ClaudeAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
https://chatgpt.com/c/67a198f6-fd28-800c-9a08-0e0dd41e3ac4

# *Required Pacakges*

In [1]:
!pip install langchain langchain_anthropic pinecone pinecone-client anthropic voyageai langchain-voyageai python-dotenv pypdf langchain-community datasets langchain_openai


In [ ]:
from google.colab import userdata
userdata.get('PINECONE_API_KEY')
userdata.get('ANTHROPIC_API_KEY')
userdata.get('VOYAGE_API_KEY')

In [ ]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    pages = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        pages.extend(loader.load())
    return pages

folder_path = "/content/"
pages = load_documents(folder_path)
print(f"Loaded {len(pages)} documents from the folder.")


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=3500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False,
)
splits = text_splitter.split_documents(pages)
print(f"split the document into  {len(splits)} chunks.")

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings = embedding_function.embed_documents([split.page_content for split in splits])

print(len(document_embeddings))

If you get error regarding index, create index manually ( dimensions= 384)


In [ ]:
# pip install langchain-pinecone

In [ ]:
import os
from langchain.vectorstores import Pinecone
from langchain_community.embeddings import HuggingFaceEmbeddings
from pinecone import Pinecone, ServerlessSpec

# Pinecone API key
pinecone_api_key = "pine_cone_api_key"
os.environ["PINECONE_API_KEY"] = pinecone_api_key


# Initialize Pinecone client
pc = Pinecone(api_key=pinecone_api_key)

# Define the index name
index_name = "agentindex"

# Check if the index exists
if index_name not in pc.list_indexes().names():
    print(f"Index '{index_name}' does not exist. Creating a new index...")
    # Create a new index with the required dimensionality (e.g., 384 for all-MiniLM-L6-v2)
    pc.create_index(
        name=index_name,
        dimension=384,  # Dimension for all-MiniLM-L6-v2 embeddings
          # Default metric for similarity search
        spec=ServerlessSpec(
            cloud="aws",  # Choose your cloud provider
            region="us-east-1"  # Choose your region
        )
    )
    print(f"Index '{index_name}' created successfully!")
else:
    print(f"Index '{index_name}' already exists.")

from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

from langchain.vectorstores import Pinecone
vectordb = Pinecone.from_documents(
    documents=splits,
    embedding=embedding_function,
    index_name=index_name,
)

print(f"Data successfully stored in Pinecone under index '{index_name}'!")

In [ ]:
# from langchain_openai import OpenAI
# from langchain.chains import RetrievalQA

# # Load an OpenAI model
# llm = OpenAI(model_name="gpt-4o-mini", api_key="")  # Replace with your API key

# # Set up the retriever (assuming vectordb is already defined)
# retriever = vectordb.as_retriever()

# # Create the RetrievalQA chain
# rag_chain = RetrievalQA.from_chain_type(
#     llm=llm,
#     retriever=retriever,
#     return_source_documents=True
# )

# print("RAG pipeline successfully set up with OpenAI model!")


In [ ]:
from transformers import pipeline
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline

# Load a free LLM from Hugging Face
model_name = "google/flan-t5-large"  # You can replace this with any other model
llm_pipeline = pipeline("text2text-generation", model=model_name, tokenizer=model_name)

# Wrap the pipeline in LangChain's HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=llm_pipeline)

# Set up the retriever (assuming vectordb is already defined)
retriever = vectordb.as_retriever()

# Create the RetrievalQA chain
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

print("RAG pipeline successfully set up with Hugging Face model!")

In [ ]:

query = "what is ai"
# Get Response from RAG System
response = rag_chain.invoke({"query": query})

# Check if response contains a valid answer
answer = response.get("result", "No relevant information found in the knowledge base.")

# Print the answer
print("Answer:", answer)
